# Home Credit - Credit Risk Model Stability

### Цель: Классификация

<div style="text-align: center;\">
    <img src="https://i.ibb.co/jDyZDf0/credit-score-med.png" width="700" height="400">
</div>

In [1]:
# Импортируем нужные библиотеки
import pandas as pd # для работы с данными
import numpy as np # для вычислений
import matplotlib.pyplot as plt # для визулизаций
import seaborn as sns # для визулизаций


from sklearn.linear_model import LogisticRegression # Линейная модель
from sklearn import preprocessing # предобработка
from sklearn import metrics # метрики
from sklearn.model_selection import train_test_split # выборка
from sklearn.ensemble import RandomForestClassifier # модель RandomForest
from sklearn.model_selection import cross_val_score, StratifiedKFold # CV

from imblearn.over_sampling import SMOTE # для дисбаланса классов

import lightgbm as lgb # модель LightGBM

import xgboost as xgb # модель XGBOOST

# Импорт пакетов для настройки гиперпараметров
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from glob import glob
from pathlib import Path
from datetime import datetime

import pickle # Сохранить модель
import gc
import dask.dataframe as dd
import warnings
from sklearn.exceptions import ConvergenceWarning

from sqlalchemy import create_engine
from tqdm.auto import tqdm
import pyarrow.parquet as pq
import re

warnings.filterwarnings('ignore', category=ConvergenceWarning)
%matplotlib inline

c:\Users\moona\Desktop\HomeCredit contest\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Повторим, как организуется процесс разработки DS-проектов согласно методологии CRISP-DM.

Этапы модели CRISP-DM:
1. Анализ требований
2. Подгрузка данных
3. Иследование и Подготовка данных
4. Моделирование
5. Оценка модели
6. Внедрение

### Анализ требований

**Нам необходимо построить модель-классификатор для вефолта кредита, то есть определить клиента который не будет платить.** Нет никаких ограничений по использовонию методов.

**Цель: F1-score был больше 0,7**

**Знакомство с данными**

Про данные можете узать по схеме - "**schema.png**". А так же можете посмотреть данные и другие решения проблемы в самой соревнований в [Kaggle](https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/overview). Так как это займет очень много место советую вам посмотреть эту [запись](https://www.kaggle.com/code/sergiosaharovskiy/home-credit-crms-2024-eda-and-submission).

У нас имеется **4 типа** таблиц:
* **Base** - Это базовая таблица с целевой переменной.
* **depth=0** - Это статические объекты, напрямую привязанные к определенному идентификатору наблюдения. (case_id)
* **depth=1** - С каждым идентификатором наблюдения связана историческая запись, индексируемая с помощью num_group1.
* **depth=2** - С каждым case_id связана историческая запись, индексируемая как num_group 1, так и num_group 2.

### Подгрузка данных

Для начала создаем **класс 'Pipeline'**, который отвечает за базовую обработку данных. Этот класс включает методы для преобразования типов данных, обработки дат и удаления ненужных столбцов на основе определённых критериев. Основная цель — подготовить данные к дальнейшему анализу или моделированию, обеспечив их консистентность и удобство работы с ними.

В этом классе я фокусируюсь на базовой обработке с помощью следующих методов:
* **Преобразование типов данных (set_table_dtypes)**: Приведение столбцов к правильным типам данных, что облегчает дальнейшую обработку и анализ.
* **Обработка дат (handle_dates)**: Преобразование дат в числовые значения, представляющие количество дней относительно ключевой даты, и удаление ненужных столбцов.
* **Удаление ненужных столбцов (basic_del_col)**: Удаление столбцов с высокой долей пропусков или низкой информативностью, чтобы улучшить качество и скорость моделирования.

In [2]:
class Pipeline:
    @staticmethod
    def set_table_dtypes(df):
        dtype_mappings = {
            "case_id": "int32",
            "WEEK_NUM": "int32",
            "num_group1": "int32",
            "num_group2": "int32",
        }
        
        for col in df.columns:
            if col in ["date_decision"]:
                df[col] = pd.to_datetime(df[col])
            elif col in dtype_mappings:
                df[col] = df[col].astype(dtype_mappings[col])
            elif col[-1] in ["P", "A"]:
                df[col] = df[col].astype("float32")
            elif col.endswith("M"):
                df[col] = df[col].astype("string")
            elif col.endswith("D"):
                df[col] = pd.to_datetime(df[col])
            elif col[-1] in ['T', 'L']:
                try:
                    df[col] = df[col].astype("float32")
                except ValueError as e:
                    df[col] = df[col].astype("string")
                    
        
        return df
    
    @staticmethod
    def handle_dates(df):
        if "date_decision" in df.columns:
            date_decision = pd.to_datetime(df["date_decision"])
            for col in df.columns:
                if col.endswith("D"):
                    df[col] = (pd.to_datetime(df[col]) - date_decision).dt.days.astype("float32")
            
            df.drop(columns=["date_decision"], inplace=True)
        
        df.drop(columns=["MONTH"], errors="ignore", inplace=True)
        
        return df
    
    @staticmethod
    def basic_del_col(df):
        cols_to_drop = []
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].isna().mean()
                if isnull > 0.8:
                    cols_to_drop.append(col)
                elif df[col].dtype == "string":
                    freq = df[col].nunique()
                    if 1 >= freq or freq > 200:
                        cols_to_drop.append(col)
        
        df.drop(columns=cols_to_drop, inplace=True)
        
        return df

In [3]:
class Aggregator:
    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        expr_max = {col: 'max' for col in cols}
        return expr_max

    @staticmethod
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D",)]
        expr_max = {col: 'max' for col in cols}
        return expr_max

    @staticmethod
    def str_expr(df):
        cols = [col for col in df.columns if col.endswith("M")]
        expr_max = {col: 'max' for col in cols}
        return expr_max

    @staticmethod
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ["T", "L"]]
        expr_max = {col: 'max' for col in cols}
        return expr_max

    @staticmethod
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = {col: 'max' for col in cols}
        return expr_max

    @staticmethod
    def get_exprs(df):
        exprs = {}
        exprs.update(Aggregator.num_expr(df)),
        exprs.update(Aggregator.date_expr(df)),
        exprs.update(Aggregator.str_expr(df)),
        exprs.update(Aggregator.other_expr(df)),
        exprs.update(Aggregator.count_expr(df))
        return exprs

Мы создаем функции для чтения данных через 'path'. Это удобно, особенно когда у нас большое количество таблиц. Функции также включают базовую обработку данных.

Создаем функцию для создания новых признаков и объединения нескольких DataFrame.

#### Конфигурация

In [4]:
ROOT            = Path("C:\\Users\\moona\\Desktop\\HomeCredit contest\\") # расположение папки
TRAIN_DIR       = ROOT / "parquet_files" / "train" # путь к данным для train
TEST_DIR        = ROOT / "parquet_files" / "test" # путь к данным для test

### Создание хранилища данных

Для удобства мы создаем хранилище в виде словаря. Как вы уже знаете, у нас есть четыре типа таблиц. У меня получилось загрузить большую часть таблиц но осталась одна, и из за проблемы с MemoryError я не смог загрузить ее. 

##### Data wrapping

In [10]:
def load_files_in_stream(result, path, depth = 0):
    try:        
        df = pd.DataFrame()
        parquet_file = pq.ParquetFile(path)
        for batch in parquet_file.iter_batches():
            df_batch = batch.to_pandas()
            
            df = pd.concat([df, df_batch], ignore_index=True)
        
        df = df.pipe(Pipeline.set_table_dtypes)
        
        if depth in [1, 2]:
            df = df.groupby("case_id").agg(Aggregator.get_exprs(df))
        
        df = Pipeline.basic_del_col(df)
        
        name = (((path.split("\\")[-1]).split("."))[0]).split("_")[-1]
        
        if "base" in path:
            result = df.copy()
            result['month_decision'] = result['date_decision'].dt.month
            result['weekday_decision'] = result['date_decision'].dt.weekday
        else:
            result = result.merge(df, how="left", on="case_id", suffixes=('', f'_{name}'))
            
        
        print(f"{path} has been successfully loaded")
    except Exception as e:
        print(f"Error loading {path}: {e}")
    del df
    gc.collect()
    return result

def load_regex_files_in_stream(result, regex_path, depth=0):
    try:
        df = pd.DataFrame()
        
        # Use glob to find files matching the regex pattern
        for path in glob(str(regex_path)):
            parquet_file = pq.ParquetFile(path)
            
            # Iterate through batches of each file
            for batch in parquet_file.iter_batches():
                df_batch = batch.to_pandas()
                
                df = pd.concat([df, df_batch], ignore_index=True)
        
        # Set appropriate dtypes
        df = df.pipe(Pipeline.set_table_dtypes)
        
        # Perform aggregation if required by depth
        if depth in [1, 2]:
            df = df.groupby("case_id").agg(Aggregator.get_exprs(df))
        
        df = Pipeline.basic_del_col(df)
        
        name = (((path.split("\\")[-1]).split("."))[0]).split("_")[-1]
        
        # Merge the current result with the new dataframe
        result = result.merge(df, how="left", on="case_id", suffixes=('', f'_{name}'))
        
        # Get the name of the last file processed for logging
        # name = (regex_path.split("\\")[-1]).split(".")[0]
        print(f"{regex_path} has been successfully loaded")
    
    except Exception as e:
        print(f"Error loading {regex_path}: {e}")
    del df
    gc.collect()
    return result
        

In [11]:
path = "parquet_files\\train\\"

data_store = {
    "df_base":  [ 
        path + "train_base.parquet"
        ],
    "depth_0": [
        path + "train_static_cb_0.parquet",
        path + "train_static_0_*.parquet"
    ],
    "depth_1": [
        path + "train_applprev_1_*.parquet",
        path + "train_other_1.parquet",
        path + "train_person_1.parquet",
        path + "train_deposit_1.parquet",
        path + "train_debitcard_1.parquet",
        path + "train_tax_registry_a_1.parquet",
        path + "train_tax_registry_b_1.parquet",
        path + "train_tax_registry_c_1.parquet",
        path + "train_credit_bureau_a_1_*.parquet",
        path + "train_credit_bureau_b_1.parquet"
    ],
    "depth_2": [
        path + "train_applprev_2.parquet",
        path + "train_person_2.parquet",
        path + "train_credit_bureau_b_2.parquet",
        # path + 'train_credit_bureau_a_2_*.parquet'
    ]
}

In [12]:
for t, path in data_store.items():
    print(t, path)

df_base ['parquet_files\\train\\train_base.parquet']
depth_0 ['parquet_files\\train\\train_static_cb_0.parquet', 'parquet_files\\train\\train_static_0_*.parquet']
depth_1 ['parquet_files\\train\\train_applprev_1_*.parquet', 'parquet_files\\train\\train_other_1.parquet', 'parquet_files\\train\\train_person_1.parquet', 'parquet_files\\train\\train_deposit_1.parquet', 'parquet_files\\train\\train_debitcard_1.parquet', 'parquet_files\\train\\train_tax_registry_a_1.parquet', 'parquet_files\\train\\train_tax_registry_b_1.parquet', 'parquet_files\\train\\train_tax_registry_c_1.parquet', 'parquet_files\\train\\train_credit_bureau_a_1_*.parquet', 'parquet_files\\train\\train_credit_bureau_b_1.parquet']
depth_2 ['parquet_files\\train\\train_applprev_2.parquet', 'parquet_files\\train\\train_person_2.parquet', 'parquet_files\\train\\train_credit_bureau_b_2.parquet']


In [13]:
result = pd.DataFrame()

In [14]:
# for t, paths in data_store.items():
#     if t in ['df_base', 'depth_0']:
#         print("First stage start:\t")
#         for path in tqdm(paths):
#             if '*' in path :
#                 result = load_regex_files_in_stream(result, path)
#             else:
#                 result = load_files_in_stream(result, path)
#     elif t in ["depth_1"]:
#         print("Second stage start:\t")
#         for path in tqdm(paths):
#             if '*' in path :
#                 result = load_regex_files_in_stream(result, path, 1)
#             else:
#                 result = load_files_in_stream(result, path, 1)
#     else:
#         print("Third stage start:\t")
#         for path in tqdm(paths):
#             if '*' in path :
#                 result = load_regex_files_in_stream(result, path, 2)
#             else:
#                 result = load_files_in_stream(result, path, 2)

First stage start:	


100%|██████████| 1/1 [00:00<00:00,  1.08it/s]


parquet_files\train\train_base.parquet has been successfully loaded
First stage start:	


 50%|█████     | 1/2 [00:07<00:07,  7.30s/it]

parquet_files\train\train_static_cb_0.parquet has been successfully loaded


...

 33%|███▎      | 1/3 [04:10<08:21, 250.86s/it]

parquet_files\train\train_person_2.parquet has been successfully loaded


 67%|██████▋   | 2/3 [13:07<06:58, 418.85s/it]

parquet_files\train\train_credit_bureau_b_2.parquet has been successfully loaded


100%|██████████| 3/3 [13:16<00:00, 265.43s/it]


In [15]:
result.to_csv("result_train.csv")

In [3]:
df_train = pd.read_csv("result_train.csv")

C:\Users\moona\AppData\Local\Temp\ipykernel_11444\509408051.py:1: DtypeWarning: Columns (8,9,15,16,19,20,24,25,67,73,75,208,227,230,233,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,323,324,325,326,327,328,329,330,331,332,333,334,354) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("result_train.csv")


In [4]:
df_train = df_train.pipe(Pipeline.handle_dates)

In [7]:
df_train.shape

(1526659, 355)

### Подготовка данных

Цель этого этапа — подготовить обучающую выборку для использования в моделировании.

**Модель:** Классификация

**Задачи:**
* Анализ дубликатов
* Удаление пустот (nan > 0.75)
* Замена пустот
* Анализ колонок по типам данных
* Удаление выбросов
* EDA
* Обработать числовые метрики
* Обработать категориальные метрики
* Удаление лишних метрик

#### Базовое работа с колонками

In [5]:
# df_train.set_index(['case_id'], inplace=True) # устанавливаем case_id как index
df_train.drop(columns='Unnamed: 0', inplace=True) # удаляем не нужную колонку
df_train.rename(columns=lambda x: re.sub(r'[ .?,]', '', x), inplace=True) # удаляем нечитаемые символы с колонок 

In [9]:
for col in df_train.columns:
    if any(char in col for char in [' ', '.', ',', '?']):
        print(col)

In [6]:
temp_data = df_train.copy() # Для удобной работы с данными

#### Анализ дубликатов

Выводим дубликаты с помощью функции duplicated() и удаляем их функцией drop_duplicates().

In [10]:
temp_data[temp_data.duplicated()] # ищем дубликаты в таблице temp_data

,case_id,WEEK_NUM,target,month_decision,weekday_decision,birthdate_574D,dateofbirth_337D,days120_123L,days180_256L,days30_165L,...,num_group2,conts_role_79M,empls_economicalst_849M,num_group1_21,num_group2_2,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A,pmts_date_1107D,num_group1_22,num_group2_21


У нас в данных не имеются дубликаты которые  могли бы нам помешать во время дальнейшей работы.

#### Удаление пустот

Выводим пустоты с колонок и удаляем колонки с пустотыми у которых пустот больше чем 75% и те колонки с типом string у которого больше 200 уникальных знечений.

In [12]:
(temp_data.isna().mean()).sort_values(ascending = False) # выводим и сортируем колонок по пустотам

residualamount_1093A       0.989438
credlmt_228A               0.989437
interestrateyearly_538L    0.986836
instlamount_892A           0.983367
numberofinstls_810L        0.983367
                             ...   
applicationcnt_361L        0.000000
applications30d_658L       0.000000
applicationscnt_1086L      0.000000
applicationscnt_464L       0.000000
applicationscnt_629L       0.000000
Length: 353, dtype: float64

In [7]:
temp_data = temp_data.pipe(Pipeline.basic_del_col)

print(temp_data.shape)

(1526659, 292)


In [12]:
(temp_data.isna().mean()).sort_values(ascending = False) # и еще раз

amount_4527230A          0.700042
num_group1_14            0.700042
recorddate_4527225D      0.700042
num_group1_16            0.684104
processingdate_168D      0.684104
                           ...   
applicationscnt_867L     0.000000
applications30d_658L     0.000000
applicationscnt_1086L    0.000000
annuity_780A             0.000000
applicationcnt_361L      0.000000
Length: 292, dtype: float64

#### Замена пустот

Для хорошего прохождения следующего этапа и хорошого результата нам нужно заменить пустоты. Я буду заменять пустоты медианным значением или модой (для данных типа object) по их классам.

In [18]:
temp_data['target'].value_counts()

target
0    1478665
1      47994
Name: count, dtype: int64

In [8]:
def fillnans(temp_data):
    classes = list(temp_data['target'].unique())
    float_int_cols = temp_data.select_dtypes(['int', 'float'])
    object_cols = temp_data.select_dtypes(['object'])
    result_temp = pd.DataFrame()
    del_cols = []
    for class_name in classes:
        temp_data_class = temp_data[temp_data['target'] == class_name]
        
        empty_columns = temp_data_class[float_int_cols].isna().mean()
        empty_columns = empty_columns[empty_columns == 1].index
        del_cols = del_cols + empty_columns
        
        empty_columns = temp_data_class[object_cols].isna().mean()
        empty_columns = empty_columns[empty_columns == 1].index
        del_cols = del_cols + empty_columns
        
        for col in temp_data_class.columns:
            if col in empty_columns:
                continue
            elif col in float_int_cols:
                temp_data_class[col].fillna(temp_data_class[col].median(), inplace=True)
            else:
                temp_data_class[col].fillna(temp_data_class[col].mode().iloc[0], inplace=True)
        
        result_temp = pd.concat([result_temp, temp_data_class], ignore_index=True)
        
        del temp_data_class
        gc.collect()
    
    result_temp.drop(columns=del_cols, inplace=True)
    result_temp = result_temp.sample(frac=1).reset_index(drop=True)
    
    return result_temp

In [9]:
del df_train

gc.collect()

0

In [10]:
temp_data_new = fillnans(temp_data)

print(f"Before: {temp_data.shape}")
print(f"After: {temp_data_new.shape}")

MemoryError: Unable to allocate 11.3 MiB for an array with shape (1478665,) and data type uint64

In [ ]:
temp_data = temp_data_new.copy()

In [ ]:
del temp_data_new

#### Анализ колонок по типам данных

##### 1) Колонки типа float

In [12]:
temp_data['num_group1_11'] = temp_data['num_group1_11'].astype('float32')

In [13]:
float_cols = list((temp_data.select_dtypes('float')).columns)
float_data = pd.DataFrame({'nunique':temp_data[float_cols].nunique(), 
              'max':temp_data[float_cols].max(), 
              'min':temp_data[float_cols].min()}, index=float_cols)

In [14]:
float_cols_1_unique = list(float_data[float_data['nunique'] == 1].index)
for col in float_cols_1_unique:
    temp_data[col] = temp_data[col].fillna(-1000).astype('int32') # указываю пустоты как -1000

In [15]:
float_cols_to_int = list(float_data[abs(float_data['max'] - float_data['min']) == float_data['nunique']].index)
for col in float_cols_to_int:
    temp_data[col] = temp_data[col].fillna(-1000).astype('int32') # указываю пустоты как -1000

In [16]:
float_cols_to_int_2_unique = list(float_data[float_data['nunique'] == 2].index)
for col in float_cols_to_int_2_unique:
    temp_data[col] = temp_data[col].fillna(-1000).astype('int32') # указываю пустоты как -1000

In [17]:
float_cols = list((temp_data.select_dtypes('float')).columns)
float_data = pd.DataFrame({'nunique':temp_data[float_cols].nunique(), 
              'max':temp_data[float_cols].max(), 
              'min':temp_data[float_cols].min()}, index=float_cols)

##### 2) Колонки типа int

In [18]:
int_cols = list((temp_data.select_dtypes('int')).columns)

##### 3) Колонки типа object

In [19]:
obj_cols = list((temp_data.select_dtypes('object')).columns)

##### Проверка:

In [20]:
len(temp_data.columns) == len(float_cols) + len(int_cols) + len(obj_cols) # Проверяем

True

#### Анализ и Удаление выбросов

В этом этапе мы будем анализировать и удалять выбросы с колонок. Это поможет нам приблизиться к более хорошому результату.